<a href="https://colab.research.google.com/github/scapolingua/Deep-Learning/blob/master/LLMfromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tokenizers datasets

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from datasets import load_dataset

# 1. Carica subset Wikipedia IT
dataset = load_dataset(
    "wikimedia/wikipedia",
    "20231101.it",
    split="train[:5000]"
)

# 2. Estrai testo
texts = [row["text"] for row in dataset]

# 3. Costruisci tokenizer BPE
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()

trainer = BpeTrainer(
    vocab_size=8000,     # PERFETTO per SLM CPU
    min_frequency=2,
    special_tokens=["<pad>", "<bos>", "<eos>", "<unk>"]
)

tokenizer.train_from_iterator(texts, trainer)

# 4. Salva tokenizer
tokenizer.save("tokenizer_it.json")

print("Tokenizer italiano BPE salvato")

In [4]:
import torch
from tokenizers import Tokenizer
from datasets import load_dataset
BLOCK_SIZE = 64

tokenizer = Tokenizer.from_file("tokenizer_it.json")

def load_wiki_tokens(limit=5000):
    dataset = load_dataset(
        "wikimedia/wikipedia",
        "20231101.it",
        split=f"train[:{limit}]"
    )

    text = "\n".join(row["text"] for row in dataset)
    tokens = tokenizer.encode(text).ids

    return torch.tensor(tokens, dtype=torch.long)


def get_batch(data, batch_size):
    ix = torch.randint(0, len(data) - BLOCK_SIZE - 1, (batch_size,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x, y


In [5]:
tokens = load_wiki_tokens(5000)
print(len(tokens))


12188185


In [7]:
model_content = """# model
#from .config import BLOCK_SIZE
import torch.nn as nn
import torch.nn.functional as F
import math
import torch

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.attn = Attention(config)
        self.ff = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class Attention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.n_head = config.n_head
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)
        self.proj = nn.Linear(config.n_embd, config.n_embd)

        self.register_buffer(
            "mask",
            torch.tril(torch.ones(config.block_size, config.block_size))
        )

    def forward(self, x):
        B, T, C = x.size()
        H = self.n_head
        k = self.key(x).view(B, T, H, C // H).transpose(1, 2)
        q = self.query(x).view(B, T, H, C // H).transpose(1, 2)
        v = self.value(x).view(B, T, H, C // H).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / math.sqrt(k.size(-1))
        att = att.masked_fill(self.mask[:T, :T] == 0, -1e9)
        att = F.softmax(att, dim=-1)

        out = (att @ v).transpose(1, 2).contiguous().view(B, T, C)
        return self.proj(out)

#------------------------------------
class GPTConfig:
    def __init__(
        self,
        vocab_size,
        block_size,
        n_layer=4,
        n_head=4,
        n_embd=128,
        dropout=0.1,
    ):
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout

# -----------------------------
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Embedding(config.block_size, config.n_embd)

        self.blocks = nn.ModuleList([Block(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        pos = torch.arange(T)
        x = self.tok_emb(idx) + self.pos_emb(pos)
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.head(x)

        if targets is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                targets.view(-1)
            )
            return logits, loss
        return logits, None
"""

with open('model.py', 'w') as f:
    f.write(model_content)

print("File model.py aggiornato.")

data_content = """import torch
from tokenizers import Tokenizer
from datasets import load_dataset

BLOCK_SIZE = 64
tokenizer = Tokenizer.from_file("tokenizer_it.json")

def load_wiki_tokens(limit=5000):
    dataset = load_dataset(
        "wikimedia/wikipedia",
        "20231101.it",
        split=f"train[:{limit}]"
    )

    text = "\\n".join(row["text"] for row in dataset)
    tokens = tokenizer.encode(text).ids

    return torch.tensor(tokens, dtype=torch.long)


def get_batch(data, batch_size):
    ix = torch.randint(0, len(data) - BLOCK_SIZE - 1, (batch_size,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x, y
"""

with open('data.py', 'w') as f:
    f.write(data_content)

print("File data.py aggiornato.")

import os
import torch
import torch.nn.functional as F
import importlib

# Create the checkpoints directory if it doesn't exist
os.makedirs('checkpoints', exist_ok=True)

# Reload the model and data modules to ensure latest changes are picked up
import model
import data
importlib.reload(model)
importlib.reload(data)
from model import GPT, GPTConfig
from data import load_wiki_tokens, get_batch

# CONFIG
BATCH_SIZE = 16
MAX_STEPS = 5000
LR = 3e-4
DEVICE = "cpu"

# DATA
data = load_wiki_tokens(limit=5000)

# MODEL
config = GPTConfig(
    vocab_size=12000,
    block_size=128,
    n_layer=4,
    n_head=4,
    n_embd=256,
)

model = GPT(config).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# TRAIN LOOP
for step in range(1, MAX_STEPS + 1):
    x, y = get_batch(data, BATCH_SIZE)
    x, y = x.to(DEVICE), y.to(DEVICE)

    logits, loss = model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print(f"Step {step} | Loss {loss.item():.4f}")

    if step % 1000 == 0:
        torch.save(
            {
                "model": model.state_dict(),
                "config": config.__dict__,
                "step": step,
            },
            f"checkpoints/slm_step_{step}.pt"
        )


File model.py aggiornato.
File data.py aggiornato.
Step 100 | Loss 7.2220
Step 200 | Loss 7.1434
Step 300 | Loss 7.0658
Step 400 | Loss 6.9124
Step 500 | Loss 6.7943
Step 600 | Loss 6.7243
Step 700 | Loss 6.7489
Step 800 | Loss 6.6171
Step 900 | Loss 6.3845
Step 1000 | Loss 6.2078
Step 1100 | Loss 6.2035
Step 1200 | Loss 6.1577
Step 1300 | Loss 6.0184
Step 1400 | Loss 6.1968
Step 1500 | Loss 6.1180
Step 1600 | Loss 5.8781
Step 1700 | Loss 5.7804
Step 1800 | Loss 5.9131
Step 1900 | Loss 5.8444
Step 2000 | Loss 5.5914
Step 2100 | Loss 5.8490
Step 2200 | Loss 5.6483
Step 2300 | Loss 6.0386
Step 2400 | Loss 5.6167
Step 2500 | Loss 5.6673
Step 2600 | Loss 5.5264
Step 2700 | Loss 5.4311
Step 2800 | Loss 5.5736
Step 2900 | Loss 5.5394
Step 3000 | Loss 5.3187
Step 3100 | Loss 5.4358
Step 3200 | Loss 5.4304
Step 3300 | Loss 5.3063
Step 3400 | Loss 5.2417
Step 3500 | Loss 5.4086
Step 3600 | Loss 5.4485
Step 3700 | Loss 5.2901
Step 3800 | Loss 5.2434
Step 3900 | Loss 5.3512
Step 4000 | Loss 5.072

In [10]:
import torch
from tokenizers import Tokenizer
from model import GPT, GPTConfig

# Carica il tokenizer
tokenizer = Tokenizer.from_file("tokenizer_it.json")

# Carica il checkpoint del modello
checkpoint_path = "checkpoints/slm_step_5000.pt"
checkpoint = torch.load(checkpoint_path, map_location='cpu')

# Ricostruisci la configurazione del modello
config = GPTConfig(**checkpoint['config'])

# Inizializza il modello e carica lo stato
model = GPT(config)
model.load_state_dict(checkpoint['model'])
model.eval() # Imposta il modello in modalità valutazione
print(f"Modello caricato con successo dal checkpoint: {checkpoint_path}")

Modello caricato con successo dal checkpoint: checkpoints/slm_step_5000.pt


In [13]:
def generate_text(model, tokenizer, prompt, num_generate=100, temperature=1.0):
    model.eval()
    # Codifica il prompt
    encoded_prompt = tokenizer.encode(prompt).ids
    context = torch.tensor(encoded_prompt, dtype=torch.long, device='cpu').unsqueeze(0)

    generated_tokens = []
    for _ in range(num_generate):
        # Se il contesto supera la block_size, troncalo
        if context.size(1) > model.config.block_size:
            context = context[:, -model.config.block_size:]

        # Ottieni i logits
        logits, _ = model(context)
        # Prendi l'ultimo timestep e applica la temperatura
        logits = logits[:, -1, :] / temperature
        # Applica softmax per ottenere le probabilità
        probs = torch.nn.functional.softmax(logits, dim=-1)
        # Campiona dalla distribuzione
        next_token = torch.multinomial(probs, num_samples=1)

        # Aggiungi il token generato alla sequenza
        context = torch.cat((context, next_token), dim=1)
        generated_tokens.append(next_token.item())

    # Decodifica la sequenza completa
    full_sequence = encoded_prompt + generated_tokens
    decoded_text = tokenizer.decode(full_sequence)
    return decoded_text

# Esempio di generazione di testo
prompt = "L'intelligenza artificiale"
generated_output = generate_text(model, tokenizer, prompt, num_generate=200, temperature=0.7)
print("--- Testo generato ---")
print(generated_output)
print("----------------------")

--- Testo generato ---
L ' intelli genza artificiale , il teorema di un periodo di un modello di coper tura di piante delle terre stri zioni . N et ano sono in cui la radiazione e l ' umani tà del gruppo permette di elementi della specie . La produzione di « Di re la scelta stra di una gestione del gruppo del più mar co . M ono no stro . È è una delle di proce dura di misura re del con una l ' si ha detto " porta più di ma sche ma nero , data base di pro pa c ' allu se co di tra ina tradizionale o stra ona rie o to di una ri uscita negli d ' unica , un totale di un numero di ma aff in particolare A ere tto , ma ga , che , al qu er na za e pe ma sul di " o non più o posto un ficato nato , in certi e R come gli A che , come di ogni tor nato che , i me do m che spa ma v eli cità di una o tri o A spe tto di sto ma in foto dal fo
----------------------


In [ ]:
prompt_new = "Il tempo è un concetto astratto e spesso difficile da definire, ma la sua importanza nella vita quotidiana è innegabile."
generated_output_new = generate_text(model, tokenizer, prompt_new, num_generate=200, temperature=0.7)
print("--- Testo generato con nuovo prompt ---")
print(generated_output_new)
print("-------------------------------------")

In [1]:
from datasets import load_dataset
import torch

# 1. Carica un subset di validazione di Wikipedia IT
# Usiamo una porzione diversa del dataset per la validazione
validation_dataset = load_dataset(
    "wikimedia/wikipedia",
    "20231101.it",
    split="train[5000:6000]" # Esempio: usiamo 1000 esempi successivi
)

# 2. Estrai il testo e tokenizzalo usando il tokenizer addestrato
validation_texts = [row["text"] for row in validation_dataset]

# Unisci il testo per la tokenizzazione come fatto per l'addestramento
validation_tokens_ids = tokenizer.encode("\n".join(validation_texts)).ids

# Converti in tensore PyTorch
validation_data = torch.tensor(validation_tokens_ids, dtype=torch.long)

print(f"Numero totale di token nel set di validazione: {len(validation_data)}")


NameError: name 'tokenizer' is not defined

In [9]:
def calculate_perplexity(model, data, batch_size, block_size, device='cpu'):
    model.eval() # Imposta il modello in modalità valutazione
    total_loss = 0.0
    total_tokens = 0

    # Non è necessario 'no_grad()' qui per lo stesso motivo che ho dato prima.
    # Con 'no_grad' le operazioni verranno eseguite in modo più efficiente in termini di memoria e velocità,
    # in quanto non sarà necessario calcolare i gradienti.
    with torch.no_grad():
        # Itera sui dati di validazione in batch
        # Un modo semplice per creare batch sequenziali per la valutazione
        for i in range(0, len(data) - block_size - 1, block_size):
            x = data[i:i+block_size].unsqueeze(0).to(device) # Input del modello
            y = data[i+1:i+block_size+1].unsqueeze(0).to(device) # Target

            logits, loss = model(x, y)

            total_loss += loss.item() * x.size(1) # Accumula la loss pesata per la lunghezza della sequenza
            total_tokens += x.size(1)

    if total_tokens == 0:
        return float('inf') # Evita divisione per zero

    average_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(average_loss))
    return perplexity.item()

# Calcola la perplessità
# Assicurati che BLOCK_SIZE del modello e della data siano gli stessi
model_block_size = model.config.block_size
perplexity = calculate_perplexity(model, validation_data, BATCH_SIZE, model_block_size, DEVICE)

print(f"Perplessità del modello sul set di validazione: {perplexity:.4f}")


Perplessità del modello sul set di validazione: 309.3773


In [15]:
prompt_long = "L'esplorazione spaziale, con i suoi incredibili progressi e le sfide che ancora attendono, cattura l'immaginazione di milioni di persone in tutto il mondo. Dalla corsa allo spazio degli anni '60 alla prospettiva di colonizzare Marte, l'umanità ha sempre guardato alle stelle con meraviglia e ambizione. Ogni nuova scoperta ci avvicina alla comprensione dell'universo, ma solleva anche nuove domande sulla nostra esistenza e sul nostro posto nel cosmo. La tecnologia avanza a passi da gigante, rendendo possibili missioni sempre più audaci e sofisticate. Quali segreti verranno svelati in futuro dalle profondità dello spazio?"
generated_output_long = generate_text(model, tokenizer, prompt_long, num_generate=250, temperature=0.8)
print("--- Testo generato con prompt più lungo ---")
print(generated_output_long)
print("------------------------------------------")

--- Testo generato con prompt più lungo ---
L ' esplo razione spaziale , con i suoi incre di bili progressi e le sfi de che ancora atten dono , cattura l ' immagina zione di milioni di persone in tutto il mondo . Dalla corsa allo spazio degli anni ' 60 alla prospe ttiva di coloni zzare Marte , l ' umani tà ha sempre guar dato alle stelle con m era vi glia e ambi zione . Ogni nuova scoperta ci av vicina alla compren sione dell ' universo , ma solle va anche nuove doman de sulla nostra esistenza e sul no stro posto nel co smo . La tecnologia avanza a passi da gigante , ren dendo possibili missioni sempre più au da ci e so fi stica te . Qu ali segre ti ver ranno s vel ati in futuro dalle profondità dello spazio ? I sto me ti vità sud classi ta x Monte cittadino e tor nata in tedesco e me ga ria a A tta , ma Sa pi è una era della cu cerca del gruppo mo a to mba , come la di cre o con l ' anti bano del ministro e mble ma che i ma i nella te me teori te x o più o ch di di un del DNA qui mb m

In [2]:
import torch
import torch.nn.functional as F
import importlib
import os

# Create the checkpoints directory if it doesn't exist
os.makedirs('checkpoints', exist_ok=True)

# Reload the model and data modules to ensure latest changes are picked up
import model
import data
importlib.reload(model)
importlib.reload(data)
from model import GPT, GPTConfig
from data import load_wiki_tokens, get_batch

# CONFIG
BATCH_SIZE = 16
MAX_STEPS = 10000 # Increase MAX_STEPS for continued training example
LR = 3e-4
DEVICE = "cpu"

# DATA
data = load_wiki_tokens(limit=5000)

# MODEL
# Initialize with a default config, will be overwritten if loading checkpoint
config = GPTConfig(
    vocab_size=12000,
    block_size=128,
    n_layer=4,
    n_head=4,
    n_embd=256,
)

model = GPT(config).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
start_step = 1

# Load checkpoint if it exists
checkpoint_path = "checkpoints/slm_step_5000.pt"
if os.path.exists(checkpoint_path):
    print(f"Caricamento checkpoint da {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    # Re-instantiate config from checkpoint to ensure consistency
    config = GPTConfig(**checkpoint['config'])
    model = GPT(config).to(DEVICE)
    model.load_state_dict(checkpoint['model'])
    start_step = checkpoint['step'] + 1
    # Optionally load optimizer state
    # optimizer.load_state_dict(checkpoint['optimizer'])
    print(f"Addestramento ripreso dal passo {start_step}")
else:
    print("Nessun checkpoint trovato, inizio addestramento da zero.")

# TRAIN LOOP
for step in range(start_step, MAX_STEPS + 1):
    x, y = get_batch(data, BATCH_SIZE)
    x, y = x.to(DEVICE), y.to(DEVICE)

    logits, loss = model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print(f"Step {step} | Loss {loss.item():.4f}")

    if step % 1000 == 0:
        torch.save(
            {
                "model": model.state_dict(),
                "config": config.__dict__,
                "step": step,
            },
            f"checkpoints/slm_step_{step}.pt"
        )


ModuleNotFoundError: No module named 'model'

In [20]:
prompt_long = "L'esplorazione spaziale, con i suoi incredibili progressi e le sfide che ancora attendono, cattura l'immaginazione di milioni di persone in tutto il mondo. Dalla corsa allo spazio degli anni '60 alla prospettiva di colonizzare Marte, l'umanità ha sempre guardato alle stelle con meraviglia e ambizione. Ogni nuova scoperta ci avvicina alla comprensione dell'universo, ma solleva anche nuove domande sulla nostra esistenza e sul nostro posto nel cosmo. La tecnologia avanza a passi da gigante, rendendo possibili missioni sempre più audaci e sofisticate. Quali segreti verranno svelati in futuro dalle profondità dello spazio?"
generated_output_low_temp = generate_text(model, tokenizer, prompt_long, num_generate=250, temperature=0.5)
print("--- Testo generato con prompt lungo (temperature=0.5) ---")
print(generated_output_low_temp)
print("----------------------------------------------------------")

--- Testo generato con prompt lungo (temperature=0.5) ---
L ' esplo razione spaziale , con i suoi incre di bili progressi e le sfi de che ancora atten dono , cattura l ' immagina zione di milioni di persone in tutto il mondo . Dalla corsa allo spazio degli anni ' 60 alla prospe ttiva di coloni zzare Marte , l ' umani tà ha sempre guar dato alle stelle con m era vi glia e ambi zione . Ogni nuova scoperta ci av vicina alla compren sione dell ' universo , ma solle va anche nuove doman de sulla nostra esistenza e sul no stro posto nel co smo . La tecnologia avanza a passi da gigante , ren dendo possibili missioni sempre più au da ci e so fi stica te . Qu ali segre ti ver ranno s vel ati in futuro dalle profondità dello spazio ? Z uri è la ma a una del sistema di una di un ga me tter ma ce m elo o per i con sor dio o di un ' a una delle più o in un cla endo per i server , e i bile in fi du i dio , la è uno , per i dio di una o il f at ch , che di che in una , ma glia ta , che ( o più o mi m

In [19]:
prompt_long = "L'esplorazione spaziale, con i suoi incredibili progressi e le sfide che ancora attendono, cattura l'immaginazione di milioni di persone in tutto il mondo. Dalla corsa allo spazio degli anni '60 alla prospettiva di colonizzare Marte, l'umanità ha sempre guardato alle stelle con meraviglia e ambizione. Ogni nuova scoperta ci avvicina alla comprensione dell'universo, ma solleva anche nuove domande sulla nostra esistenza e sul nostro posto nel cosmo. La tecnologia avanza a passi da gigante, rendendo possibili missioni sempre più audaci e sofisticate. Quali segreti verranno svelati in futuro dalle profondità dello spazio?"
generated_output_long = generate_text(model, tokenizer, prompt_long, num_generate=250, temperature=0.8)
print("--- Testo generato con prompt più lungo e modello aggiornato ---")
print(generated_output_long)
print("----------------------------------------------------------")

--- Testo generato con prompt più lungo e modello aggiornato ---
L ' esplo razione spaziale , con i suoi incre di bili progressi e le sfi de che ancora atten dono , cattura l ' immagina zione di milioni di persone in tutto il mondo . Dalla corsa allo spazio degli anni ' 60 alla prospe ttiva di coloni zzare Marte , l ' umani tà ha sempre guar dato alle stelle con m era vi glia e ambi zione . Ogni nuova scoperta ci av vicina alla compren sione dell ' universo , ma solle va anche nuove doman de sulla nostra esistenza e sul no stro posto nel co smo . La tecnologia avanza a passi da gigante , ren dendo possibili missioni sempre più au da ci e so fi stica te . Qu ali segre ti ver ranno s vel ati in futuro dalle profondità dello spazio ? ( l ' 11 bi zione della dice f na scente tto ine i ferra mentare ), e del ca una spin it ma mmi che le fu un va inven zione della bi ologia del cor dia della classe proveni to da pa ta mento con la ma ris pose a bo tte nel che , i ma che la cui in seguito al 